# Export Psi4 as OLDMOS

* Run Psi4
* Convert MOs (reordering, scaling)
* Export as `PSIMOS`

This notebook demonstrates the process step-by-step by calling the low-level functions of the library.

In [4]:
import psi4
import numpy as np
import sys
sys.path.append('../../../lib')
from P4toC4_aux import *

Run Psi4 with Cartsian coordiantes that will **not**
be reoriented in either Psi4 or CFOUR.

The original input was:

```
O
H 1 0.96
H 1 0.96 2 104.5
```

In [5]:
psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", "CC-PVDZ")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("REFERENCE", "RHF")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

h2o = psi4.geometry("""
0 1
O     0.00000000   0.00000000  -0.12429784
H     0.00000000  -1.43441957   0.98634846
H     0.00000000   1.43441957   0.98634846
symmetry c1
units Bohr
no_reorient
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-76.02665365047244

In [3]:
# Use this basis set as GENBAS to guarantee best results
# basisset=wf.basisset()
# print(basisset.genbas())

Creates mapping from Psi4 to CFOUR:
```
Cfour_MO[mapping[i]] = Psi4_MO[i] / scale[i]
```

In [4]:
mapping, scale = basis_mapping(wf.basisset(), verbose=0)
#mapping

Applies the mapping and writes MOs in CFOUR format.

In [5]:
# Psi4 MOs
Ca=np.array(wf.Ca())
Cb=np.array(wf.Cb())
Ca_C4 = psi4_to_c4(Ca, mapping, scale)
Cb_C4 = psi4_to_c4(Cb, mapping, scale)
write_oldmos('PSIMOS', Ca_C4, Cbs=Cb_C4)

Move the file PSIMOS to your CFOUR work-directory
(for this example, `Cfour/RHF`), and run the script `runscf`.